# Introduction

This notebook presents an implementation of an **Article Writer app using an agentic approach** composed by an agent with two LLM-based tools:

- **Research Tool**: A tool that queries the DuckDuckGo search API and is useful for when you need to answer questions about current events.

- **Writing Tool**: A tool that writes text by leveraging an LLM and is useful for when you need to write an engaging text based on a set of insights.

The code is created using the LangChain library, the OpenAI LLM API, and the DuckDuckGo search API.

**By the end of the notebook, it is discussed the need for using a multi-agent system for this task.**

In [1]:
#! pip install --quiet \
#              duckduckgo-search==6.3.0 \
#              langchain==0.2.14 \
#              langchain-community==0.2.10 \
#              langchain-core==0.2.32 \
#              langchain-openai==0.1.20 \
#              python-dotenv==1.0.1

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.tools import tool
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_openai import ChatOpenAI

In [3]:
use_google_drive_api = False
google_drive_path = 'gdrive/My Drive/Profissional/Haleon/Talks/Multi Agent vs Single Agent with LLM-based Tools/code'

if(use_google_drive_api):
  from google.colab import drive
  drive.mount('/content/gdrive/', force_remount=True)
  # Changing the current working directory to the notebook folder
  os.chdir(google_drive_path)
  print(os.getcwd())

In [4]:
# OpenAI API Key

_ = load_dotenv(find_dotenv()) # read local .env file
api_key = os.environ["OPENAI_API_KEY"]

In [5]:
# Instantiate an LLM

llm = ChatOpenAI(
    api_key=api_key,
    temperature=0.4,
    model_name="gpt-4o",
)

In [6]:
# Instantiate the agent tools

# The research tool

search_tool = DuckDuckGoSearchRun()

In [7]:
type(search_tool)

langchain_community.tools.ddg_search.tool.DuckDuckGoSearchRun

In [8]:
# The writing tool

system_instructions = """
You are a Large Language Model (LLM) designed to assist in the creation of high-quality written content. Your primary function is to generate well-structured, engaging, and coherent text based on the insights, guidelines, and parameters provided by the user. Here's how you should approach your tasks:

1. Understand the Input:
- Analyze Insights: Carefully read and comprehend the key points, outlines, or any other input provided. Ensure you fully understand the content, context, and goals before generating text.
- Determine Purpose: Identify the purpose of the content—whether it’s to inform, persuade, entertain, or explain—and tailor your output accordingly.

2. Content Generation:
- Tone and Style: Produce content that matches the specified tone and style, whether formal, informal, conversational, technical, or creative. Ensure consistency throughout the text.
- Structure and Coherence: Organize the content logically, with a clear flow from introduction to conclusion. Use appropriate headings, subheadings, and transitions to enhance readability.
- Engagement: Focus on making the content engaging and compelling. Use varied sentence structures, relevant examples, and a dynamic writing style to capture the reader's attention.

3. Customization and Iteration:
- Follow Guidelines: Adhere strictly to any specific instructions regarding length, structure, or content focus. Ensure that all key points are covered comprehensively.
- Refinement: After generating the initial draft, be prepared to refine the text. Address any feedback or adjustments needed to improve clarity, accuracy, or overall quality.
- Adaptation: Be flexible in adjusting the content as required by the user, including changing the tone, rephrasing sections, or expanding on specific ideas.

4. Final Review:
- Quality Check: Review the entire content to ensure it meets high standards of grammar, coherence, and engagement. Correct any errors and polish the text to be publication-ready.
- Completion: Ensure the final output fully aligns with the user’s requirements and is ready for delivery or further processing.

Your goal is to generate text that not only meets the user's specifications but also stands out for its quality, readability, and impact.
"""

chat_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(system_instructions),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)

writer_chain = chat_prompt | llm

@tool
def writing_tool(insights: str):
  """
  Useful for when you need to write engaging blog posts based on the provided insights.
  Input should be the insights gathered.
  """

  response = writer_chain.invoke(insights)

  return response.content

In [9]:
# Define tool belt

tools = [search_tool, writing_tool]

In [10]:
# Instantiate the agent

system_instructions = """
You are an AI article writer responsible for crafting high-quality articles based on user requests. Your task is to generate informative and engaging content by strategically utilizing two specialized tools: a research tool and a writing tool. Here's how you should approach each task:

1. Understand the User's Request: Carefully analyze the theme and requirements provided by the user. Determine the key topics and questions that need to be addressed in the article.

2. Research and Information Gathering:
- When to Use: Utilize the research tool when you need to gather insights on current events or specific queries where up-to-date information is crucial.
- How to Use: Perform comprehensive research to collect relevant, accurate, and reliable data. Review the information gathered to ensure it covers all aspects needed for the article.

3. Content Creation:
- When to Use: Use the writing tool to compose the article, focusing on creating engaging and well-structured content based on the insights obtained.
- How to Use: Draft the article with clear, coherent, and captivating writing. Ensure the content is tailored to the audience and adheres to the user’s specified tone and style.

4. Execution and Iteration:
- Plan Your Workflow: Before starting, outline the structure of the article, decide the sequence of tasks, and determine how you will use the tools at each stage.
- Review and Refine: After generating content, review it for accuracy, coherence, and engagement. Use the tools iteratively to refine and enhance the article until it meets high standards of quality.
- Assess Completion: Ensure that the article fully addresses the user's request, answers all key questions, and is polished for final delivery.

Your goal is to deliver a well-researched, informative, and engaging article that exceeds user expectations.
"""

chat_prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(system_instructions),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad", optional=True),
    ]
)

agent = create_openai_tools_agent(
    llm, tools, chat_prompt
)

return_intermediate_steps = True
verbose = True 

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    handle_parsing_errors=True,
    return_intermediate_steps=return_intermediate_steps,
    verbose=verbose,
).with_config({"run_name": "Agent"})

In [15]:
# Invoke the agent to write an article about AI in 2024

input = "The most significant AI advancements in 2024."

response = agent_executor.invoke({"input": input})

print("-----------------------------")
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'most significant AI advancements in 2024'}`


Adobe Stock. It's been a year since OpenAI released ChatGPT, opening the door to seamlessly weave AI into the fabric of our daily lives, propelling industries into the future and even prompting ... The most important AI trends in 2024 Tags Artificial Intelligence 9 February 2024 12 min read. Link copied 2022 was the year that generative artificial intelligence (AI) exploded into the public consciousness, and 2023 was the year it began to take root in the business world. 2024 thus stands to be a pivotal year for the future of AI, as ... NVIDIA's (NASDAQ:NVDA) GPUs such as the A100 and H100 Tensor Core GPU are at the forefront of AI and machine learning. In the fiscal year 2024, NVIDIA's (NASDAQ:NVDA) revenue increased 53% ... In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering

In [16]:
# Check the intermediate steps taken by the agent

print('Intermediate Steps:')
for index, tool_action_output in enumerate(response['intermediate_steps']):
  action = tool_action_output[0]
  print(f"   {index}: call {action.tool} with {action.tool_input}")

Intermediate Steps:
   0: call duckduckgo_search with {'query': 'most significant AI advancements in 2024'}
   1: call writing_tool with {'insights': 'In 2024, AI technology continues to make significant strides, influencing various sectors and everyday life. One of the notable advancements is the increased accessibility and utility of generative AI for the general public. This year, generative AI models are becoming more user-friendly, allowing non-tech individuals to leverage AI capabilities in their daily tasks. Additionally, the development and deployment of open-source, fully capable small language models that can operate on personal devices like phones and laptops mark a significant shift towards personalized and accessible AI solutions.\n\nNVIDIA remains at the forefront of AI and machine learning advancements, with their A100 and H100 Tensor Core GPUs driving significant growth in AI capabilities. In fiscal year 2024, NVIDIA reported a 53% increase in revenue, underscoring the 

# Discussion: Is it necessary a multi-agent system?

In the proposed approach, the agent works more like an editor (or manager/supervisor, in a general way) once it does not write the article per se, but instead, the agent plans the execution, uses the appropriate tools to perform tasks, reviews the outputs, and assesses task completion. The logical diagram is shown in the figure below:

<p align="center">
  <img src="./img/article_writer_via_single_agent_with_llm_based_tools.png" alt="" width="300">
</p>

<center>Image source: Author.</center>

On the other hand, **this interpretation is conceptually wrong** once a tool must be used for someone. Whoever is using a writing tool to write something is a writer, not an editor.

To overcome this logical failure, the natural extension of this approach is a multi-agent system, where each tool is a specialized agent, and the current agent is indeed an editor (manager/supervisor) to route work between different agents. The logical diagram is shown in the figure below:

<p align="center">
  <img src="./img/article_writer_via_multi_agent_sequential_without_delegation_using_crewai.png" alt="" width="330">
</p>

<center>Image source: Author.</center>

Is the interpretation indeed the unique advantage? No! Conceptually, a tool works in a linear way, while an agent can use several tools and reflect on or interact with the result before returning the outcome. Of course, in the single agent approach, the agent can call the same tools several times, as illustrated in the study case; however, it does not have a specific instruction in order to be able to evaluate the tool outcome as a specialist. For simple cases, it must not make any difference, but for complex scenarios, a multi-agent approach may be more effective, as in the example of [Hierarchical Agent Teams](https://github.com/langchain-ai/langgraph/blob/a16a33806c60e80be1fb64b54f2e5835f6080c01/examples/multi_agent/hierarchical_agent_teams.ipynb) example from LangGraph:

<p align="center">
  <img src="./img/hierarchical-diagram.png" alt="" width="600">
</p>

<center>Image source: LangGraph - Hierarchical Agent Teams.</center>